# Grandline

This tutorial will guild you through the following steps:

1. Load synthetic linear data (200 samples, 5000 genes/nodes, 20 clusters) and Create adjacency matrix `A`.
    - `cv_data_dict[i]['X_train']` (160 sample x 5000 node)
    - `cv_data_dict[i]['X_test']` (40 sample x 5000 node)
    - `cv_data_dict[i]['y_train']` (160 sample x 1)
    - `cv_data_dict[i]['y_test']` (40 sample x 1)
    - `i=0,1,...,9` (10 shuffles)

2. Set GCN hyperparameters :
`epoch, learning rate, regularization, batch_size, number of graph convolutional filters(Fs), polynomial orders(Ks), pooling sizes(Ps), fully connected layers(Ms)`

3. Train model

4. Make a prediction

5. Grad-CAM
    - Calculate node important for each sample
    - Summarize node important across samples for each class

In [1]:
import pandas as pd
import numpy as np
import networkx as nx 
import scipy
import pickle, os
import seaborn as sns
import tensorflow as tf

from lib import  graph, coarsening, utils, grandline

Note: The GPU is not needed here as our sample dataset is small and will take only few minute on CPU. To use GPU, please check https://www.tensorflow.org/guide/gpu for more detail. Grandline is implemented using Tensorflow 2.0.0.

In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

## Read data and preprocessing

In [3]:
disease = 'synthetic'
disease_type = 'X_linear'
input_name = 'RandomPartition_5000_20'
input_prefix = 'data/{}_{}'.format(disease_type, input_name)

cv_data_dict = pickle.load(open("{}_cv.pickle".format(input_prefix), "rb"))
n_shuffle = 10

In [4]:
gene_list = cv_data_dict[0]['X_train'].columns
d = len(gene_list)

print ("Number of genes", d)
print ("List of genes", gene_list)

Number of genes 5000
List of genes Index(['N00000', 'N00001', 'N00002', 'N00003', 'N00004', 'N00005', 'N00006',
       'N00007', 'N00008', 'N00009',
       ...
       'N04990', 'N04991', 'N04992', 'N04993', 'N04994', 'N04995', 'N04996',
       'N04997', 'N04998', 'N04999'],
      dtype='object', length=5000)


In [5]:
temp_df = cv_data_dict[0]['y_train']
C = temp_df.groupby(temp_df.columns[0]).size().shape[0]
print ("Number of classes", C)

Number of classes 2


### Change DataFrame to numpy array and reshape

In [6]:
current_shuffle = 0

In [7]:
for name in ['X_train', 'X_test']:    
    cv_data_dict[current_shuffle][name]= cv_data_dict[current_shuffle][name].values.astype(np.float32)

for name in ['y_train', 'y_test']:    
    cv_data_dict[current_shuffle][name] = cv_data_dict[current_shuffle][name].values.astype(np.uint8)
        

cv_data_dict[current_shuffle]['y_test'] = cv_data_dict[current_shuffle]['y_test'].reshape((cv_data_dict[current_shuffle]['y_test'].shape[0],))
cv_data_dict[current_shuffle]['y_train'] = cv_data_dict[current_shuffle]['y_train'].reshape((cv_data_dict[current_shuffle]['y_train'].shape[0],))

In [8]:
X_train = cv_data_dict[current_shuffle]['X_train']
y_train = cv_data_dict[current_shuffle]['y_train']
X_test = cv_data_dict[current_shuffle]['X_test']
y_test = cv_data_dict[current_shuffle]['y_test']

### Create adjacency matrix A

In [9]:
A = utils.prepare_adjacency('data/A_{}.csv'.format(input_name), gene_list)
print ("Created A {}x{}".format(A.shape[0], A.shape[1]))

Created A 5000x5000


## GCN hyperparameters


In [10]:
params = dict()
params['num_epochs']     = 15
params['learning_rate']  = 1e-3
params['filter_name']    = 'chebyshev'

seed = 8

params['Fs']              = [20, 20]  # Number of graph convolutional filters. 
params['Ks']              = [10, 10]  # Polynomial orders.
params['Ps']              = [2, 2]  # Pooling sizes. 
params['Ms']              = [C]  # Output dimensionality of fully connected layers.

params['regularization'] = 1e-5
params['batch_size'] = X_train.shape[0]


### Calculate normalized laplacian (L) for each level

In [11]:
n_level_coarsen = int(np.log2(params['Ps']).sum())
print ("Coarsening level:", n_level_coarsen)
Ls, graphs, perms = graph.calculate_laplacian(A, levels=n_level_coarsen)

Coarsening level: 2
Layer 0: M_0 = |V| = 5032 nodes (32 added),|E| = 307436 edges
Layer 1: M_1 = |V| = 2516 nodes (5 added),|E| = 218625 edges
Layer 2: M_2 = |V| = 1258 nodes (0 added),|E| = 123181 edges


### Arrange features/genes according to permutation (from coarsening)

In [12]:
if perms is not None:
    X_train = coarsening.perm_data(X_train, perms[0])
    X_test = coarsening.perm_data(X_test, perms[0])

In [13]:
from tensorflow.keras.utils import to_categorical 
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [14]:
X_train = np.expand_dims(X_train, 2)
X_test = np.expand_dims(X_test, 2)

X_train.shape, X_test.shape

((160, 5032, 1), (40, 5032, 1))

## Train GCN model

In [15]:
tf.keras.backend.clear_session()

In [16]:
model, model_logit = grandline.build_gcn_model(graphs, Ls, **params)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate'], name='Adam'), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
              metrics=['accuracy'])
model.build(input_shape=X_train.shape)

### Call back functions

In [17]:
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                                      min_delta=0.001, 
                                                      patience=15,
                                                      verbose=1,
                                                      mode='max',
                                                      baseline=None, 
                                                      restore_best_weights=True)

### Define class weight

In [18]:
from sklearn.utils import class_weight

y_train_class_name = np.argmax(y_train, axis=1)
class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(y_train_class_name),
                                                  y_train_class_name)
class_weights = dict(enumerate(class_weights))
class_weights

{0: 1.0, 1: 1.0}

### Start model training

In [19]:
history = model.fit(x=X_train,
                    y=y_train,
                    epochs=params['num_epochs'],
                    validation_data=[X_test, y_test],
                    batch_size=params['batch_size'],
                    class_weight=class_weights,
                    callbacks=[earlystop_callback], # checkpoint_callback
                    verbose=1, shuffle=True)

Train on 160 samples, validate on 40 samples
Epoch 1/15
160/160 [==============================] - 10s 63ms/sample - loss: 0.7290 - accuracy: 0.4938 - val_loss: 0.5427 - val_accuracy: 0.5250
Epoch 2/15
160/160 [==============================] - 9s 54ms/sample - loss: 0.5442 - accuracy: 0.5188 - val_loss: 0.1159 - val_accuracy: 1.0000
Epoch 3/15
160/160 [==============================] - 9s 55ms/sample - loss: 0.1315 - accuracy: 1.0000 - val_loss: 0.1064 - val_accuracy: 1.0000
Epoch 4/15
160/160 [==============================] - 9s 54ms/sample - loss: 0.1453 - accuracy: 1.0000 - val_loss: 0.0693 - val_accuracy: 1.0000
Epoch 5/15
160/160 [==============================] - 8s 53ms/sample - loss: 0.0990 - accuracy: 1.0000 - val_loss: 0.0229 - val_accuracy: 1.0000
Epoch 6/15
160/160 [==============================] - 8s 52ms/sample - loss: 0.0336 - accuracy: 1.0000 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 7/15
160/160 [==============================] - 8s 53ms/sample - loss: 0.0095 

## Make a prediction 

In [20]:
logit_model = tf.keras.Model(inputs=model_logit.inputs, outputs=model_logit.outputs)

In [21]:
predict = model.predict(x=X_test)
print ("Predicted values:")
np.argmax(predict, axis=1)

Predicted values:


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [22]:
print ("True values:")
np.argmax(y_test, axis=1)

True values:


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Grad-CAM

### Calculate node important for each sample

`cal_gradcam` function returns node importance of the last covolutional layer.

In [23]:
num_train = X_train.shape[0]

sample_label = []
node_importance = []

for selected_sample_id in range(num_train):
    X_input = np.expand_dims(X_train[selected_sample_id], 0).astype('float32')
    sample_label += [np.argmax(y_train[selected_sample_id])]
    node_importance += [grandline.cal_gradcam(selected_sample_id, X_input, logit_model)]

Create DataFrame for node importance

In [24]:
importance_allnode_df = pd.DataFrame(np.array(node_importance))
node_columns = importance_allnode_df.columns
importance_allnode_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515
0,-0.001457,-0.000658,-0.000733,-0.000727,0.003130,0.001816,0.004044,0.002066,0.000154,-0.001147,...,-0.001599,-0.005290,0.007975,-0.000044,-0.001514,-0.000064,0.002345,-0.000007,0.000932,-0.000126
1,-0.001471,-0.001410,-0.001095,0.000314,0.002567,0.001859,0.000670,0.001257,-0.000360,-0.001298,...,-0.001241,-0.004373,0.002956,-0.000044,-0.002076,-0.000064,0.003270,-0.000007,0.001820,-0.000126
2,-0.001803,-0.001020,-0.001258,-0.000016,0.003903,0.001099,0.004608,0.001858,-0.000683,-0.001354,...,-0.000704,-0.002962,0.006517,-0.000044,0.001108,-0.000064,0.001586,-0.000007,0.001122,-0.000126
3,-0.001731,-0.001297,-0.002513,-0.000407,0.004001,0.001575,0.001283,0.001939,-0.002006,-0.001421,...,-0.002811,-0.006517,0.009819,-0.000044,-0.001728,-0.000064,0.007483,-0.000007,0.000288,-0.000126
4,-0.001476,-0.001170,-0.000524,-0.003466,0.001153,0.000913,0.003735,0.002841,-0.000832,-0.001133,...,0.000979,-0.005068,0.006627,-0.000044,-0.000374,-0.000064,0.004039,-0.000007,0.000554,-0.000126


Infer node important values to each node in the input graph. Here we consider absolute important values.

In [25]:
importance_allnode_df = importance_allnode_df.abs()

In [26]:
sample_node_important_df_list = []

for i in range(num_train):
    sample_node_important_df = utils.get_node_importance_df(perms, importance_allnode_df.loc[i, node_columns], d).set_index('node')[['important']]
    sample_node_important_df.columns = ['train_{}'.format(i)]
    sample_node_important_df_list += [sample_node_important_df]
    
ipt_df = pd.concat(sample_node_important_df_list, axis=1).reset_index()
ipt_df.index = gene_list
ipt_df = ipt_df.drop('node', axis=1)
ipt_df.head()

,train_0,train_1,train_2,train_3,train_4,train_5,train_6,train_7,train_8,train_9,...,train_150,train_151,train_152,train_153,train_154,train_155,train_156,train_157,train_158,train_159
N00000,0.000213,0.003232,0.000543,0.000522,0.002714,0.002878,0.004862,0.004200,0.005491,0.001164,...,0.000943,0.002220,0.001092,0.001653,0.001697,0.000617,0.000501,0.000055,0.001606,0.001673
N00001,0.001194,0.002905,0.001239,0.002255,0.000218,0.001857,0.002158,0.001485,0.001985,0.002365,...,0.004047,0.002359,0.004383,0.001917,0.004368,0.001851,0.003362,0.003968,0.004877,0.004391
N00002,0.005489,0.002887,0.001718,0.002107,0.003082,0.003859,0.006862,0.000435,0.001886,0.003601,...,0.000447,0.000724,0.001299,0.001686,0.001146,0.001637,0.003153,0.001022,0.002315,0.001346
N00003,0.001857,0.005526,0.002899,0.006016,0.004776,0.005589,0.006640,0.005094,0.005745,0.007854,...,0.000585,0.001389,0.000712,0.001740,0.000578,0.001012,0.000027,0.000147,0.001668,0.000145
N00004,0.005242,0.005293,0.004814,0.005816,0.006061,0.003648,0.003380,0.006086,0.006143,0.004513,...,0.001608,0.000435,0.001387,0.000877,0.000852,0.002105,0.001764,0.000553,0.001297,0.000452


### Summarize node important across samples for each class

In [27]:
ipt_df = ipt_df.T
ipt_df.loc[:, 'label'] = sample_label

In [28]:
summary_ipt_df = ipt_df.groupby(['label']).sum().T
summary_ipt_df.columns = ['label_0', 'label_1']
summary_ipt_df.index.name = 'Id'
summary_ipt_df.columns.name = ''
summary_ipt_df

,label_0,label_1
Id,,
N00000,0.227807,0.110702
N00001,0.154467,0.250200
N00002,0.298304,0.124068
N00003,0.429635,0.114606
N00004,0.411817,0.091072
...,...,...
N04995,0.257853,0.271609
N04996,0.185106,0.106518
N04997,0.259140,0.217495
